# 2. Data Wrangling

### Table of contents  

- Pandas Dataframes
   - Creation
   - Properties
   - Accessing
   - Extension
   - Missing values
   - Deletion
   - Operations
      - Arithmetic
      - Logical
      - String Functions
      - Arbitrary Functions
      - Statistical
      - Ordering
   - Data Transformations
      - Group By
      - Pivot
      - Melt
      - Join
   - Time Series
   - Hierarchical Indices
   - Loading and Saving Dataframes


- Further feading

## 2b. Pandas Dataframes

Dataframes are easy-to-use, functionality-packed data structures for data handling and analysis. They can be thought of as non-homogenous matrices  for row/column labeled data, which also offer a lot of extra functionality.

In [ ]:
import numpy as np
import pandas as pd  # the usual abbreviation

### Creation

Instantiate from a 2D array:

In [ ]:
m = [
    [5, 2, 3],
    [4, 5, 1],
    [7, 1, 2],
    [6, 2, 9],
]

In [ ]:
pd.DataFrame(m)

When working with labeled data, provide a dict of `column_name : value_for_each_row`:

In [ ]:
students = pd.DataFrame({
    'height':    np.random.randint(150, 200, size=5),
    'weight':    np.random.randint(50,  100, size=5),
    'graduated': np.random.random(size=5) > .5,  # same as np.random.randint(0, 1, size=10).astype(bool)
})

Each row is an observation (i.e.: a student), each column is a variable (i.e.: a measurement):

In [ ]:
students

_Note_: since randomness is involved, when you run this notebook, you'll likely see different results.

Dataframes are implicitly indexed by integers, but rows can be assigned more descriptive indices:

In [ ]:
students.index = list('abcde')

In [ ]:
students

**💪 Exercise**: create a new dataframe, `food_stats`, which contains ratings, on a 1-3 scale for `tasty`, `healthy` and whether you had it recently (`had_recently`) for the following types of food: `pizza`, `carrot`, `chocolate`, `banana`:

### Properties

Shape and data type are extended to dataframes as well:

In [ ]:
len(students)  # size of first dimension, i.e.: number of rows

In [ ]:
students.shape

In [ ]:
students.dtypes

In [ ]:
students.height.astype(float)

**💪 Exercise**: check the `shape` and `dtypes` of your `food_stats`:

### Accessing

Access rows elements by index:

In [ ]:
students

In [ ]:
students.loc['a']  # the observations for student A

Access rows by their position, regardless of index name:

In [ ]:
students.iloc[0]

Slices are extended row-wise:

In [ ]:
students[:3]

In [ ]:
students.head(3)

---

Access a specific column:

In [ ]:
students['graduated']

**Tip**: sometimes, reserved keys cannot be used to access certain columns (e.g.: `size`), so you must use the dict-like syntax

In [ ]:
students.graduated  # we can also access column-wise

Access multiple columns at once:

In [ ]:
columns = ['weight', 'height']
students[columns]

Masking is performed row-wise by providing a boolean array with the same length as the number of rows:

In [ ]:
students[students.graduated]

---

Randomly sample rows:

In [ ]:
students.sample(3)  # 3 rows

In [ ]:
students.sample(frac=.5)  # half of all rows

In [ ]:
len(students)

---

Iterating over the dataframe defaults to going over its columns:

In [ ]:
for column in students:
    print(column)

`iterrows()` is used to iterate over each row:

In [ ]:
for student, row in students.iterrows():
    print(student, row.height, row.weight)

**💪 Exercise**: access the `tasty` column, rows 2 through 4 of `food_stats`:

**💪 Exercise**: select rows for students yet to graduate:

### Extension

To add a new row, just specify it's index (`loc`) or positional index (`iloc`) and the value for each column:

In [ ]:
students.loc['x'] = (170, 70, True)

In [ ]:
students

**ℹ️ Tip**: even though `df.iloc[len(df) - 1] = ...` can be used to append at the end of any dataframe, this is not recommended. If you wish to create a dataframe iteratively, instead of appending each element, store them in a different container and convert the data to a dataframe at the end. This is also the reason why there is no `append` function for dataframes.

To add a new column, assign it directly to the dataframe and specify a value for each row:

In [ ]:
students['age'] = np.random.randint(18, 24, size=len(students))

In [ ]:
students

**ℹ️ Tip**: this syntax must be used for column creation. `df.column` only works for accessing existing columns.

Create a column based on another:

In [ ]:
students['can_ride'] = (students.height > 170)  # "you must be this tall to ride the roller coaster"

In [ ]:
students

---

Add multiple new rows, from another dataframe:

In [ ]:
new_students = pd.DataFrame({
    'height': [160, 180],
    'weight': [ 60,  80],
})

new_students

In [ ]:
students = pd.concat([students, new_students], sort=False)
students

_Note_ it's `pd.concat` but `np.concatenate`

Add multiple new columns, from another dataframe:

In [ ]:
n_students = len(students)

new_info = pd.DataFrame({
    'fav_number':   np.random.randint(0, 100, size=n_students),
    'fav_icecream': np.random.choice(['vanilla', 'chocolate', 'strawberry'], size=n_students),
})

In [ ]:
new_info.index = students.index  # set the same index, to make merging (and viewing) easier
new_info

In [ ]:
students = students.merge(new_info, left_index=True, right_index=True)
students

**💪 Exercise**: add a new entry for `hamburger` in the `food_stats` dataframe:

### Missing Values

Since we only provided the `height` and `weight` measurements, for the new students, `nan` (not a number) is placed by default for the other columns:

In [ ]:
students

Missing value detection:

In [ ]:
pd.isna(students)

Column-wise:

In [ ]:
pd.isna(students.age)

The easiest method for handling missing data is dropping the observation alltogether:

In [ ]:
students.dropna()

Another method is filling it with some default value:

In [ ]:
students.fillna({
    'graduated': False,
    'can_ride':  False,
    'age':       20,
})

More methods for handling missing value will be explored in workshop 4.

**ℹ️ Tip**: the reason why we must use `pd.isna` to identify NANs, instead of `== np.nan` is because `np.nan` is a special element, which, by design, is not equal to anything, not even `np.nan`. Read more about [ternary logic](https://en.wikipedia.org/wiki/Three-valued_logic) and non-[finite numpy numbers](https://docs.scipy.org/doc/numpy-1.15.1/reference/generated/numpy.isfinite.html#numpy.isfinite).

### Deleting

Delete some columns:

In [ ]:
students.drop(['weight', 'age'], axis=1)

Delete some rows, index-wise:

In [ ]:
students.drop(['a', 'x'], axis=0)

---

Deleting rows based on a boolean filtering is done by masking:

In [ ]:
mask = (students.age > 21)  # filter underage ones
students[mask]

Dropping duplicate rows can be done using `drop_duplicate`, but since our dataframe contains no such rows, we will restrict it to duplicates on just the `can_ride` and `fav_icecream` columns:

In [ ]:
students

In [ ]:
students.drop_duplicates(subset=['can_ride', 'fav_icecream'], keep='first')

---

Deleting, much like any operation in the next subsection, does not operate _in place_. This means the `drop` function returns a new dataframe object, which is created by (deep) copying the original one and applying the operation on it. Writing your own functions in such a way helps with preventing unexpected and hard to trace side effects. It also allows for method chaining `df.transpose().mean().round()`. Read more about [functional programming](https://hackernoon.com/learn-functional-python-in-10-minutes-to-2d1651dece6f) and [immutability in Python](https://www.pythonforthelab.com/blog/mutable-and-immutable-objects/).

If you do wish to "update" the same object, assign the result to the same variable:

In [ ]:
students = students.dropna()

Some functions also offer the `inplace` option:

In [ ]:
students.dropna(inplace=True)  # equivalent to above

**💪 Exercise**: Drop the `graduated` column from `students` (not in place!):

### Dataframe Operations

Array-wise functions and operations are naturally extended to dataframes

**ℹ️ Tip**: a dataframe is composed by multiple `pd.Series`. Each column can be a series, and each row can be a series. A series is a "labeled list" — where each `value` has an `index`.

In [ ]:
students

#### Arithmetic

In [ ]:
students.weight - 10  # if only losing weight was this easy 😅

In [ ]:
students.height + students.weight  # note that the resulting series still has the same indices

In [ ]:
students.sum()

In [ ]:
students.mean()

**ℹ️ Tip**: The sum of the boolean series `graduated` is the number of people that graduated. The mean of `graduated` is the sum divided by the total number of students, which is precisely the percentage of students that graduated.

Most operations accept an `axis` argument, which can either be `0` (column-wise, default), or `1` (row-wise):

In [ ]:
students.mean(axis=1)  # the average for each student, of their height, weight and graduation status.. which doesn't make much sense

**💪 Exercise**: get the sums of the `tasty` and `healthy` ratings in `food_stats`:

#### Logical

In [ ]:
students.age > 21

In [ ]:
students.graduated & ~students.can_ride  # students who graduated but cannot ride

In [ ]:
students.graduated.any()

In [ ]:
students.graduated.all()

In [ ]:
students == 21

**💪 Exercise**: which tasty (`tasty` score $\ge 2$) food items have you had recently?

#### String Functions

The raw string values of textual variables can be accessed with `.str`:

In [ ]:
students.fav_icecream

In [ ]:
students.fav_icecream.str.title()

In [ ]:
students.fav_icecream.str.replace('straw', 'banned-')

In [ ]:
students.fav_icecream.str.contains('e')

**💪 Exercise**: in `food_stats`, make the name of the food items (the `index`) UPPERCASE:

#### Arbitrary Functions

In [ ]:
students.height.apply(lambda w: (w // 10) * 10)  # apply to each element of a column

In [ ]:
students.apply(lambda row: row.height + row.weight, axis=1)  # apply row-wise

In [ ]:
def relabel_boolean(x):
    # if the argument is not a boolean, leave it as it is
    if type(x) is not bool:
        return x
    return 'yes' if x is True else 'no'

In [ ]:
students.applymap(relabel_boolean)  # apply element-wise

#### Statistical

One of the most useful shortcuts is `describe`, which quickly provides a list of descriptive statistics about each numeric column:

In [ ]:
students.describe().round(2)

90% of the values are lower than the 90th percentile. This can be used for outlier detection, which will be explored in workshop 4.

In [ ]:
students.describe(percentiles=[.1, .5, .9, .95]).round(3)

---

Categorical variables aggregations:

In [ ]:
students.fav_icecream.value_counts()

In [ ]:
students.fav_icecream.unique()

In [ ]:
students.fav_icecream.nunique()  # when you need just the amount of unique items, not their actual values

---

Two variables have high _pair-wise correlation_ (Pearson) when one is dependent on the other: there is a linear relationship between the two:

In [ ]:
students.corr()

Two variables have high _covariance_ when they then to show similar behavior: greater values in one correspond to greater values in the other:

In [ ]:
students.cov()

Positive _skewness_ indicates left-leaning distributions:

In [ ]:
students.age.skew()

Positive _kurtosis_ indicates that the distribution has heavy tails and sharp peaks:

In [ ]:
students.age.kurt()

Read more about [distribution measures](https://support.minitab.com/en-us/minitab-express/1/help-and-how-to/basic-statistics/inference/supporting-topics/data-concepts/how-skewness-and-kurtosis-affect-your-distribution/).

**💪 Exercise**: `describe` your `food_stats`:

#### Ordering

Sort by the index:

In [ ]:
students

In [ ]:
students.sort_index()

Sort by (a combination of) column(s):

In [ ]:
students.sort_values(by=['fav_number', 'age'], ascending=True)

Compute each row's rank (average ranks in case of equality):

In [ ]:
students.fav_number.rank()

**💪 Exercise**: sort your `food_stats` by `tasty`est first:

### Data Transformations

Restructuring operations. Same data, but different view, more fitted for the downstream task.

Transpose rows and columns (keeping labels):

In [ ]:
students.T

Transform a categorical variable into dummy variables:

In [ ]:
pd.get_dummies(students.fav_icecream)

### Group By

_Grouping_ puts together rows according to the values for a certain variable:

In [ ]:
# exemplify on a new dataframe
performance = pd.DataFrame([
    ('Alice', 'CS 101', 4.0),
    ('Alice', 'CS 102', 3.0),
    ('Alice', 'EE 201', 4.0),
    ('Bob',   'CS 101', 3.0),
    ('Bob',   'EE 201', 3.9),
], columns=['student', 'class', 'grade'])

In [ ]:
performance

In order to see the effects of grouping, we apply an aggregation on all the rows for each student:

In [ ]:
performance.groupby('student').grade.mean()

---

Group by multiple variables:

In [ ]:
students.groupby(['graduated', 'fav_icecream']).age.mean()

---

Iterate over the `groupby` object, feature value (student name here) and the rows for that value:

In [ ]:
for student, classes in performance.groupby('student'):
    print(student, 'took', len(classes), 'classes, with an average of', classes.grade.mean().round(2))

**💪 Exercise**: get the maximum grade of each student in the `performance` dataframe:

### Pivot

_Pivoting_ "flips" the data according and applies an function. Select a discrete variable for the columns, one for the rows, and you get the unique values for each. Each observations in the original dataframe, falls into one such value intersection. Pick an aggregation to apply to that set of observations.

For example, if want to know the average `height` and `weight` for those students that `graduated` and those that did not:

In [ ]:
pd.pivot_table(
    students,
    index='graduated',
    values=['height', 'weight'],
    aggfunc='mean',
)

Or the maximum `height` and `weight` instead:

In [ ]:
pd.pivot_table(
    students,
    index='graduated',
    values=['height', 'weight'],
    aggfunc='max',
)

---

A special case of pivoting is _cross tabulation_, which returns the counts at each feature value intersection:

In [ ]:
pd.crosstab(students.fav_icecream, students.graduated, margins=True)

**💪 Exercise**: get `mean` of `age` and `height` for students who `can_ride` and those who can't:

### Melt

_Melting_ can be thought of as the reverse of pivoting.

In [ ]:
# exemplify on a new dataframe
height_evolution = pd.DataFrame({
    'Alice': np.linspace(160, 190, num=5),
    'Bob':   np.linspace(170, 180, num=5),
    'year':  range(2000, 2005),
})

In [ ]:
height_evolution

Currently, each student has their own column. But that is the same type of information. So, we can melt them into a single `student` column:

In [ ]:
melted = height_evolution.melt(
    id_vars='year',
    value_vars=['Alice', 'Bob'],

    var_name='student',
    value_name='height',
)
melted

### Join

_Joining_ combines two tables, based on a common variable

In [ ]:
height_stats = height_evolution.describe().T[['min', 'max']].rename(columns=lambda c: c + '_height')
height_stats

_Note_: since these are column-wise statistics, we also get the smallest and largest values for `year`.

Both tables have the `student` column, which is the one we join on. For every row where `student` is `"Alice"`, combine the information from the `performance` dataframe (`class` and `grade`) with the information from the the other dataframe (`min_height` and `max_height`).

In [ ]:
performance.join(height_stats, on='student')

_Note_: since `year` is not among `performance.student` values, the default left join ignores that entry. Learn more about [join types](http://www.sql-join.com/sql-join-types/).

### Time Series

In [ ]:
from datetime import datetime

In [ ]:
# number of pages read for the first week of classes
pages_read = pd.DataFrame({
    'Alice': np.random.randint(0, 50, size=7),
    'Bob':   np.random.randint(0, 20, size=7),
    'date': pd.date_range('7 Jan 2019', periods=7)
})

pages_read

Comparison operations can be done against a `datetime`-compatible object:

In [ ]:
late_start = datetime(year=2019, month=1, day=10)  # it's not fair to start counting that early

In [ ]:
pages_read[pages_read.date > late_start]

While dates represent specific timepoints (of various granularity), the difference between two such objects is a _time delta_: a duration, not a date:

In [ ]:
late_start - pages_read.date.iloc[0]

It can be instantiated by parsing natural language:

In [ ]:
pd.Timedelta('7 days 5 hours 3 minutes')

Timedeltas can be used to offset date objects:

In [ ]:
pages_read.date + pd.Timedelta(7, 'd')  # much better, a whole week later

### Hierarchical Indices

In [ ]:
enrollment = pd.DataFrame({
    'level':    np.random.choice(['grad', 'undergrad', 'phd'], size=20),
    'school':   np.random.choice(['Viterbi', 'Price', 'Marshall', 'Dornsife'], size=20),
    'students': np.random.randint(200, 5_000, size=20),
    'faculty':  np.random.randint(50,  500,   size=20),
}).drop_duplicates(subset=['level', 'school'])

enrollment

In [ ]:
enrollment.set_index(['school', 'level']).sort_index()

---

In [ ]:
locations = pd.DataFrame({
    'Alice': ('San Francisco', 'CA', 'Los Angeles', 'CA'),
    'Bob':   ('Rochester', 'NY', 'Los Angeles', 'CA'),
    'Chris': ('Las Vegas', 'NV', 'Pennsylvania', 'PA'),
}).T

locations.columns = pd.MultiIndex.from_product([
    ['home', 'school'],
    ['city', 'state']
], names=['purpose', 'address'])

locations

Read more about [advanced indexing](https://pandas.pydata.org/pandas-docs/stable/user_guide/advanced.html)

### Loading and Saving Dataframes

Data will almost always be loaded from an external source.

Load from JSON (open the file in the file browser to check out the source):

In [ ]:
pd.read_json('example_files/objects.json')

Load from CSV (again, the file is in the `example_files` folder):

In [ ]:
pd.read_csv('example_files/tabular.csv')

**ℹ️ Tip**: the CSV format is extremely common. There is a huge number of options available for loading such files. Read more about them [here](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html).

Load directly from an URL, letting Pandas do the downloading (paste that link into your browser to check the source):

In [ ]:
pd.read_csv('https://raw.githubusercontent.com/stefan-niculae/viz-workshop/master/example_files/tabular.csv')  # can take a second to load

Load directly from an archive, letting Pandas do the uncompressing (find the file and extract it in your local file browser to check the source):

In [ ]:
pd.read_csv('example_files/archived.csv.zip')

---

Saving data (check the result after running in the file browser):

In [ ]:
performance.to_csv('students_performance.csv')

**ℹ️ Tip**: if the index is meaningless (e.g.: just the default sequential one), avoid wasting space and slightly encumbering the reading process by omitting it with `ignore_index=True`.

## Further reading
 - Pandas:
   - [visual cheatsheet](http://pandas.pydata.org/Pandas_Cheat_Sheet.pdf)
   - [cookbook](https://chrisalbon.com/#python)
   - [gotchas](https://pandas.pydata.org/pandas-docs/stable/user_guide/gotchas.html#gotchas)
 
Links to more details about particular concepts are placed at the end of their respective (sub)sections.